In [1]:
from tensorflow import keras
import tensorflow as tf
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import urllib.request
import time
import os
import torch
from matplotlib import pyplot as plt

#### 1. 웹에서 이미지 크롤링하기

In [2]:
# def scroll_down(driver):
#     last_height = driver.execute_script("return document.body.scrollHeight")
#     while True:
#         # 페이지 맨 아래로 스크롤
#         driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
#         time.sleep(2)
#         new_height = driver.execute_script("return document.body.scrollHeight")
#         if new_height == last_height:
#             try:
#                 load_more_button = driver.find_element(By.XPATH, '//input[@value="더보기"]')
#                 if load_more_button.is_displayed():
#                     load_more_button.click()
#                     time.sleep(2)
#             except:
#                 break
#         last_height = new_height

# query = input("검색어: ")
# image_cnt = int(input("수집할 이미지 개수: "))

# url = 'https://search.naver.com/search.naver?ssc=tab.image.all&where=image&sm=tab_jum&query=%EB%B9%84%EC%88%91+%ED%94%84%EB%A6%AC%EC%A0%9C+%ED%86%A0%EB%A6%AC'


# os.chdir('D:\\physiognomy\\dsf')

# driver = webdriver.Chrome()  # Chrome 웹 드라이버 실행
# driver.get(url)  # 검색어를 포함한 URL로 이동

# scroll_down(driver)  # 페이지 스크롤 함수 호출
# time.sleep(2)

# # 이미지 정보 추출
# soup = BeautifulSoup(driver.page_source, 'html.parser')
# image_info_list = soup.find_all('img', class_='_fe_image_tab_content_thumbnail_image')
# image_and_name_list = []

# print('=== 이미지 수집 시작 ===')

# download_cnt = 0
# for i in range(len(image_info_list)):
#     if download_cnt == image_cnt:
#         break
#     try:
#         if 'data-src' in image_info_list[i].attrs:
#             save_image = image_info_list[i]['data-src']
#         elif 'src' in image_info_list[i].attrs:
#             save_image = image_info_list[i]['src']
#         else:
#             continue
#         image_path = os.path.join(query.replace(' ', '_') + '_' + str(download_cnt+330) + '.jpg')
#         print(image_path)
#         image_and_name_list.append((save_image, image_path))
#         download_cnt += 1
#     except Exception as e:
#         print(f"이미지 URL 추출 실패: {e}")

# # 이미지 다운로드
# for i in range(len(image_and_name_list)):
#     try:
#         urllib.request.urlretrieve(image_and_name_list[i][0], image_and_name_list[i][1])
#         print(f"다운로드 완료: {image_and_name_list[i][1]}")
#     except Exception as e:
#         print(f"이미지 다운로드 실패: {e}")

# print('=== 이미지 수집 종료 ===')

#### 2. 데이터 전처리

- 위의 크롤링한 사진 중에 강아지만 인식해서 파일을 가져옴
- 불필요한 특성 추출 방지를 위해서 객체 인식 방식을 선택함

In [3]:
# # YOLOv5 모델 로드
# os.chdir('D:\\physiognomy')

# model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

# for i in os.listdir('D:\\physiognomy\\dsf'):
#     img_path = 'D:\\physiognomy\\dsf\\' + i # 여기에 이미지 경로를 입력하세요.
#     img = cv2.imread(img_path)

#     # 이미지를 모델에 입력
#     results = model(img)

#     # 결과를 DataFrame으로 변환
#     df = results.pandas().xyxy[0]

#     # 'name' 열에서 'dog' 라벨 필터링
#     dog_df = df[df['name'] == 'dog']

#     # 개 인식 결과 출력
#     print(dog_df)

#     # 개가 인식되지 않은 경우 처리
#     if dog_df.empty:
#         print("No dogs detected in the image.")
#     else:
#         # 여러 개의 개가 있을 경우 첫 번째 개를 선택 (여러 개를 모두 자르고 싶다면 루프 사용)
#         first_dog = dog_df.iloc[0]

#         # 경계 상자의 좌표
#         xmin, ymin, xmax, ymax = int(first_dog['xmin']), int(first_dog['ymin']), int(first_dog['xmax']), int(first_dog['ymax'])

#         # 이미지 자르기
#         cropped_img = img[ymin:ymax, xmin:xmax]

#         # 자른 이미지 표시
#         cv2.imshow('dog',cv2.cvtColor(cropped_img, cv2.COLOR_BGR2RGB))
#         cv2.waitKey(5)
#         cv2.destroyAllWindows()

#         # 자른 이미지 저장 (선택 사항)
#         cv2.imwrite('D:\\physiognomy\\dog_face\\'+i, cropped_img)

In [2]:
def data(path):
    os.chdir(path)
    files = os.listdir(path)
    images = [cv2.imread(os.path.join(path, i),cv2.IMREAD_GRAYSCALE) for i in files]
    resized_images = [cv2.resize(img, (120,120), interpolation=cv2.INTER_LINEAR) for img in images]
    x = np.array(resized_images)
    return x

In [3]:
file_path = 'D:\\physiognomy\\dog_face'
file_names = os.listdir(file_path)

y = []

for i in file_names:
    if 'bi' in i:
        y += [1]
    elif 'pome' in i:
        y += [2]
    elif 'pug' in i:
        y += [3]
    elif 'siberian_' in i:
        y += [4]
    elif 'chiwawa_' in i:
        y += [5]
    elif 'golden_' in i:
        y += [6]

In [4]:
x = data("D:\\physiognomy\\dog_face")
y = np.array(y)

In [5]:
np.unique(y,return_counts=True)

(array([1, 2, 3, 4, 5, 6]), array([210, 194, 266, 202, 229, 232], dtype=int64))

In [9]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y,random_state=100, test_size=0.2)

In [10]:
print(np.unique(ytrain,return_counts=True))
print(np.unique(ytest,return_counts=True))

(array([1, 2, 3, 4, 5, 6]), array([168, 153, 213, 166, 177, 189], dtype=int64))
(array([1, 2, 3, 4, 5, 6]), array([42, 41, 53, 36, 52, 43], dtype=int64))


In [8]:
model = keras.Sequential()

model.add(keras.layers.Conv2D(120,kernel_size=3,activation='relu',padding='same',input_shape=(120,120,1)))
model.add(keras.layers.MaxPooling2D(2))
model.add(keras.layers.Conv2D(240,kernel_size=(3,3),activation='relu',padding='same'))
model.add(keras.layers.Dropout(0.4))
model.add(keras.layers.MaxPooling2D(2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(120, activation='relu'))
model.add(keras.layers.Dropout(0.4))
model.add(keras.layers.Dense(60, activation='relu'))
model.add(keras.layers.Dense(7, activation='softmax'))

model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 120, 120, 120)     1200      
                                                                 
 max_pooling2d (MaxPooling2  (None, 60, 60, 120)       0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 60, 60, 240)       259440    
                                                                 
 dropout (Dropout)           (None, 60, 60, 240)       0         
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 30, 30, 240)       0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 216000)           

In [11]:
os.chdir('D:\\physiognomy')

In [15]:
# 데이터 정규화
xtrain_scaled = xtrain/255
xtest_scaled = xtest/255

In [13]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics='accuracy')
cb = keras.callbacks.ModelCheckpoint('cnn_model.hdf5')
es_cb = keras.callbacks.EarlyStopping(patience=2,restore_best_weights=True)
h = model.fit(xtrain_scaled,ytrain,epochs=15,
              callbacks=[cb,es_cb])


Epoch 1/15


34/34 [==============================] - ETA: 0s - loss: 2.0851 - accuracy: 0.2092

c:\Users\amysm\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


34/34 [==============================] - 198s 6s/step - loss: 2.0851 - accuracy: 0.2092
Epoch 2/15
34/34 [==============================] - 192s 6s/step - loss: 1.5311 - accuracy: 0.4090
Epoch 3/15
34/34 [==============================] - 194s 6s/step - loss: 1.3257 - accuracy: 0.4747
Epoch 4/15
34/34 [==============================] - 199s 6s/step - loss: 1.1745 - accuracy: 0.5366
Epoch 5/15
34/34 [==============================] - 181s 5s/step - loss: 1.0487 - accuracy: 0.5966
Epoch 6/15
34/34 [==============================] - 180s 5s/step - loss: 0.9233 - accuracy: 0.6463
Epoch 7/15
34/34 [==============================] - 161s 5s/step - loss: 0.7836 - accuracy: 0.7008
Epoch 8/15
34/34 [==============================] - 168s 5s/step - loss: 0.6640 - accuracy: 0.7505
Epoch 9/15
34/34 [==============================] - 183s 5s/step - loss: 0.4931 - accuracy: 0.8293
Epoch 10/15
34/34 [==============================] - 183s 5s/step - loss: 0.3857 - accuracy: 0.8659
Epoch 11/15
34/34 [=

In [14]:
evaluate_result = model.evaluate(xtest_scaled, ytest)
evaluate_result

9/9 [==============================] - 10s 999ms/step - loss: 1.2976 - accuracy: 0.6142


[1.2975951433181763, 0.6142321825027466]

In [16]:
os.chdir('D:\\physiognomy')

In [17]:
m =  tf.keras.models.load_model('cnn_model.hdf5')
m.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 120, 120, 120)     1200      
                                                                 
 max_pooling2d (MaxPooling2  (None, 60, 60, 120)       0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 60, 60, 240)       259440    
                                                                 
 dropout (Dropout)           (None, 60, 60, 240)       0         
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 30, 30, 240)       0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 216000)            0

In [18]:
print(m.evaluate(xtrain_scaled, ytrain))
print(m.evaluate(xtest_scaled, ytest))


34/34 [==============================] - 32s 923ms/step - loss: 0.2663 - accuracy: 0.9296
[0.26632580161094666, 0.9296435117721558]
9/9 [==============================] - 9s 966ms/step - loss: 0.4068 - accuracy: 0.8914
[0.4067699909210205, 0.8913857936859131]


In [23]:
img = cv2.imread('seonggi.jpg',cv2.IMREAD_GRAYSCALE)
img = cv2.resize(img, (120,120), interpolation=cv2.INTER_LINEAR)
img_scaled = (img / 255).reshape(120,120,1)
img_scaled = np.expand_dims(img_scaled, axis=0)

print(m.predict(img_scaled))
predicted_class = np.argmax(m.predict(img_scaled), axis=1)
predicted_class

dic = {1: '너는 비글 닮음', 2: '너는 포메 닮음', 3: '너는 퍼그 닮음', 4: '너는 허스키 닮음',5:'너는 치와와 닮음',6:'너느 골든 리트리버 닮음'}
print("Predicted class:", dic[predicted_class[0]])

1/1 [==============================] - 0s 92ms/step
[[1.1348886e-08 1.8937330e-01 1.0388881e-06 1.8899280e-04 2.1644415e-02
  7.8845108e-01 3.4109954e-04]]
1/1 [==============================] - 0s 86ms/step
Predicted class: 너는 치와와 닮음
